# Planet Quota API Introduction

This notebook exists to highlight basic use and functionality of [Planet's](https://www.planet.com) Quota Reservations API. The Quota API will allow you to create, estimate, and view existing quota reservations on the Planet platform for select products. This tutorial assumes familiarity with the [Python](https://python.org) programming language, as well as basic [REST API](https://restfulapi.net/) concepts.

## Requirements

- API Key
    - An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's, and authentication for the Quota API is handled via API key.
    - Instructions on obtaining your Planet API key can be found in the [API authentication documentation](https://docs.planet.com/develop/authentication/#obtaining-your-api-key).
- AOI Features on the Planet Platform, created with the Planet [Features API](https://docs.planet.com/develop/apis/features/) or Features Manager.

## Supported Products

- [PlanetScope](https://docs.planet.com/data/imagery/planetscope/) 
    - Select PlanetScope monitoring and access products
- [Analysis Ready PlanetScope (ARPS)](https://docs.planet.com/data/imagery/arps/)
- [Planetary Variables](https://docs.planet.com/data/planetary-variables/)


## Useful links 
* [Planet Quota API Documentation](https://docs.planet.com/develop/apis/quota/)
* [Planet Features API Documentation](https://docs.planet.com/develop/apis/features/)

____

## Set up

In order to interact with the Planet Quota API, we need to import the necessary packages and authenticate our session with our API Key.

### Imports

In [ ]:
import requests
import os

# Setup Planet Quota API base URL
QUOTA_URL = "https://api.planet.com/account/v1/quota-reservations/"

### Authentication

You should *export* your API Key as an environment variable on your system:
```bash
export PL_API_KEY="YOUR API KEY HERE"
```
Or add the variable to your path, etc. If you are using our Docker environement, it will already be set.

In [ ]:
# Authenticate with the Planet SDK for Python using your API key; See docs: https://docs.planet.com/develop/authentication
# Check for PL_API_KEY environment variable, otherwise type in your API key.
pl_api_key = os.getenv('PL_API_KEY')
if not pl_api_key:
    import getpass
    pl_api_key = getpass.getpass('Planet API Key: ')
    os.environ['PL_API_KEY'] = pl_api_key

# Setup and authenticate the session
session = requests.Session()
session.auth = (pl_api_key, '')

_____

### Basic Quota API Functionality

The most basic functionality for the Quota API is to check for existing quota reservations and what products you have available quotas for. 

First, check for any existing quota reservations you might have. If you haven't actually made a quota reservation yet, the `count` key will be zero in the results.

In [13]:
quota_res = session.get(QUOTA_URL)
print("Status code:", quota_res.status_code)

Status code: 200


In [14]:
quota_res.json()

{'meta': {'args': {'limit': 50, 'offset': 0},
  'count': 0,
  'next': None,
  'prev': None},
 'results': []}

To check for the actual products that you have avaialable to make quota reservations for, you can send a get request to the `/my/products/` endpoint. To make a quota reservation of these products, you will need the products `id` from the results of the request.
<br><br>
Note that if you're looking for subscriptions to products that don't require quota reservations (such as regular PlanetScope subscriptions), use the `/my/subscriptions/` endpoint instead.

While the product name of `PS_ARPS_SR_DAILY_ALL_TIME`, is used for all Analysis Ready PlanetScope subscriptions, the `id` will be a unique integer for you, as the assignment of the product id is by both the product and the account given access to it.

Here are the important keys your products endpoint:

- `id`: The unique identifier of how this product is assigned to you, to use as product_id in reservation requests
- `name`: The product identifier, independent of assignment (e.g., "FCM_30M")
- `title`: Human-readable product name
- `description`: Product description
- `supports_reservation`: Boolean indicating if the product supports quota reservations (only products with true can be used with reservation endpoints)
- `quota_total`: Total quota available for your organization
- `quota_used`: Quota already consumed or reserved
- `unlimited_quota`: Whether the product has unlimited quota

In [24]:
# products endpoint url
products_url = "https://api.planet.com/account/v1/my/products/"

products_res = session.get(products_url)
print("Status code:", products_res.status_code)

Status code: 200


In [25]:
products_res.json()

{'meta': {'args': {'limit': 50, 'offset': 0},
  'count': 1,
  'next': None,
  'prev': None},
 'results': [{'area': {'description': 'Minimum AOI spend.',
    'op': 'gte',
    'units': 'sqkm',
    'value': 25},
   'billable_id': 'prn:usage-api:bucket:8aa442a6-5838-4289-89e6-8e215cdfbc83',
   'created_at': '2026-01-26T18:37:57.356369',
   'description': 'Daily pre-processed, surface reflectance data prepared for analysis. All Time Archive.',
   'id': 290378,
   'name': 'PS_ARPS_SR_DAILY_ALL_TIME',
   'organization_id': 776979,
   'product_family': 'Planet Analysis Ready Data (L3)',
   'quota_total': 100000.0,
   'quota_units': 'sqkm',
   'quota_used': 0.0,
   'resource_ids': ['PS_ARD_SR_DAILY'],
   'state': 'active',
   'supports_reservation': True,
   'term_end_date': '2027-01-01T23:59:59.999999',
   'term_start_date': '2026-01-01T00:00:00',
   'title': 'Surface Reflectance ARD - Daily (All Time Archive)',
   'unlimited_quota': False,
   'updated_at': '2026-01-26T18:37:57.356377'}]}

In [26]:
# If you have access to multiple products, you can change the index number to get the desired product ID
product_response_json = products_res.json()['results'][0]

product_id = product_response_json['id']

___

### Features for Quota Reservations

When we submit requests to the Quota API, we will need a Feature Reference ID to link our request to a Feature within a Feature Collection on the Planet Platform. If you do not have any Features currently, you can learn how to use the Planet Python SDK to upload them with the [Planet Python SDK Features Demo](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/features_api/planet_sdk_features_demo.ipynb) notebook, or use the `requests` module based [Planet Features API](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/features_api/planet_features_api.ipynb) notebook.

The code below will assume you have Features already, and makes two requests to the Features API. The first will get our Feature Collections, which will include a link to the collections themselves. The second will get a collection, which will contain its items with a Feature Reference ID, `'pl:ref'`. We will set this to a variable, `pl_ref`, that we will use in our Quota API calls. If you already have a Feature Reference ID handy for a Feature that is more relevant to you, feel free to set it as the pl_ref variable and skip the calls to the Feature API.

In [29]:
FEATURE_URL = "https://api.planet.com/features/v0/ogc/my/collections"

collections = session.get(FEATURE_URL)
print("Status code:", collections.status_code)
collection_id = collections.json()['collections'][0]['id']

# 0 idx item in the collection links is the colleciton link, while the 1 idx item is the items link
collection_items_url = collections.json()['collections'][0]['links'][1]['href']
print(f"Collection ID: {collection_id}, Collection Items URL: {collection_items_url}")

Status code: 200
Collection ID: cairo_areas-Kr7JYmq, Collection Items URL: https://api.planet.com/features/v0/ogc/my/collections/cairo_areas-Kr7JYmq/items


In [30]:
# We will use the first item in the collection for this example

collection_items = session.get(collection_items_url)
pl_ref = collection_items.json()['features'][0]['properties']['pl:ref']

In [31]:
pl_ref

'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP'

___

### Quota Reservation Estimates

Now that we have a Feature Reference ID handy, we will first post a request to the `/estimate/` endpoint of the Quota API to calculate a `total_cost` for a quota reservation. The only required parameters for your reservation estimate are the product ID, which we set above in the variable `product_id, and  Feature Reference ID(s) _or_ Collection ID (more about this in the __Bulk Quota__ reservations section) and the 

*IMPORTANT* Quota Reservation post requests will return a `quota_remaining` key, which is your quota remaining at the time of the post request, _NOT_ what will be remaining from use of the estimate.

In [37]:
ESTIMATE_URL = QUOTA_URL + 'estimate'
# aoi_refs input should be a list of pl:refs
aoi_refs = [pl_ref]

estimate_payload = {
    "aoi_refs": aoi_refs,
    "product_id": product_id,
}

estimate_res = session.post(ESTIMATE_URL, json=estimate_payload)

In [38]:
estimate_res.json()

{'estimated_costs': [{'aoi_ref': 'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP',
   'cost': 25.0,
   'reserved': False}],
 'quota_remaining': 100000.0,
 'quota_total': 100000.0,
 'quota_units': 'sqkm',
 'total_cost': 25.0}

____

### Submitting Quota Reservations or Bulk Quota Reservations



In [ ]:
# Add your code here

_____